In [10]:
import awswrangler as wr
import pandas as pd
# import pymysql
# import datetime

    

# Lambda 핸들러 함수. 이벤트와 컨텍스트 매개변수를 받습니다.


# 첫 번째 SQL 쿼리 실행
sql_query_all = """
with raw as(
select
client_access_time as reg_ts
, row_number( ) over ( partition by uuid order by client_access_time asc  ) as rn
, case when json_extract_scalar(user_properties, '$.utm_medium') = 'paid' then 1 else 0 end as is_paid
, uuid
from moyo_logs.user_behavior_logs
where 1=1
    and date( date_ymd) between date( '2022-12-01') and date( '2023-07-01')
)
-- , total as (
select
date_trunc('month', reg_ts) as month
-- , is_paid
, count( distinct uuid ) as cnt
-- , count( distinct case when rn = 1 then uuid end ) as cnt_first
from raw
group by 1
Order by 1 desc
"""

df = wr.athena.read_sql_query(sql=sql_query_all, database="moyo_logs")
bucket = 'moyo-mart'
table_name = 'moyo_monthly_incremental_total'
res = wr.s3.to_parquet(
    df=df,
    path=f"s3://{bucket}/{table_name}/",
    dataset=True,
    database="moyo_mart",
    table=table_name,
    mode="overwrite_partitions",
    # partition_cols=['date_ym']
            # partition_cols 는 마지막 칼럼만 사용 가능
)

In [7]:
df

NameError: name 'df' is not defined